In [61]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util


import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup

import html 
import re
import numpy as np

class Data_utils():

    def __init__(self):

        self.original_dataframe = self.prepare_original_dataframe()

    def get_all_publis_from_database(self):
        db_df = pd.read_csv("mot.csv", sep=";",encoding="utf8", names =["idmot","motcol","niveau","identifiant","resume"])
        return db_df

    def prepare_original_dataframe(self):
        """ Prepare the original dataframe from full corpus """
        def cleanhtml(raw_html):
            cleanr = re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', raw_html)
            return html.unescape(cleantext)

        df_publis = self.get_all_publis_from_database()
        df_publis["motcol"] = df_publis.motcol.apply(lambda x: cleanhtml(BeautifulSoup(x).text))
        df_publis["resume"] = df_publis.resume.apply(lambda x: cleanhtml(BeautifulSoup(x).text) if x is not np.NaN else x)
        return df_publis



class SemanticSimilarity():

    def __init__(self):
        #self.corpus_embeddings = torch.load("./notebooks/corpus_embeddings.pt")
        self.corpus_embeddings = torch.load("./corpus_embeddings.pt")
        self.data_utils = Data_utils()
        self.data_corpus = self.prepare_original_dataframe()
        self.model = SentenceTransformer('allenai-specter')



    def search_n_articles(self,id_article):
        """ Get the n most similar articles contained in the database compute from cosine similarity 
        distance between tensors.
        Parameters:
        -----------
        id_article: int, the id of the original article  
        n: int, the number of articles returned from the search request
        Returns: List(int), list of ids of the most semantically similar articles
        """
        #TODO prétraiter le texte importé
        df_publis = self.data_utils.original_dataframe
        paper = df_publis[df_publis.idmot == id_article]
#        print(paper)
        text = paper.motcol +" "+ paper.resume

        query_embedding = self.model.encode(text.values[0], convert_to_tensor=True)

        search_articles = util.semantic_search(query_embedding, self.corpus_embeddings)[0] 
        matchs = []
        for hit in search_articles:
            related_paper = self.data_corpus.iloc[hit['corpus_id'],:]
            titre_selected = self.data_utils.original_dataframe.motcol == related_paper['motcol']
            if self.data_utils.original_dataframe[titre_selected].resume.values[0] != ";":
                matchs.append(self.data_utils.original_dataframe[titre_selected].idmot.values[0])
            
            
        
        return matchs



    def prepare_original_dataframe(self):
        """Prepare the dataframe """
        df_publis = self.data_utils.original_dataframe
        df = df_publis.loc[:,["motcol","resume","idmot"]].dropna()
        return df
        # text = df.titre + df.resume


    # def search_papers(self, text ):
    #     query_embedding = self.model.encode(text, convert_to_tensor=True)

    #     search_articles = util.semantic_search(query_embedding, self.corpus_embeddings)[0] 
    #     matchs = []
    #     for hit in search_articles:
    #         related_paper = self.data_corpus.iloc[hit['corpus_id'],:]
    #         titre_selected = self.data_utils.original_dataframe.titre == related_paper['titre']
    #         matchs.append(self.data_utils.original_dataframe[titre_selected].id.values[0])
    #         return matchs

In [62]:
model = SemanticSimilarity()

D:\A756805\Anaconda3\envs\marathon\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "data" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [63]:
ids = model.search_n_articles(11820)

In [64]:
for i in ids:
    print(i)
    print(db_df[db_df.idmot == i].loc[:,"motcol"].values[0])
    print(db_df[db_df.idmot == i].loc[:,"resume"].values[0])

10024
Quelles évaluations numériques pour l'acte de construire ?
&lt;p&gt;R&amp;eacute;sum&amp;eacute; : La num&amp;eacute;risation de l&amp;rsquo;acte de construire est en cours. Qu&amp;rsquo;il s&amp;rsquo;agisse de concevoir les b&amp;acirc;timents, de les r&amp;eacute;aliser ou de les g&amp;eacute;rer, la mutation num&amp;eacute;rique interroge les acteurs de l&amp;rsquo;habitat. Elle s&amp;rsquo;incarne dans un acronyme : le BIM pour Building Information Model, Modeling et Management. Impuls&amp;eacute; par une directive europ&amp;eacute;enne en 2014, le BIM se d&amp;eacute;veloppe tranquillement dans les diff&amp;eacute;rentes organisations charg&amp;eacute;es de l&amp;rsquo;acte de construire ainsi que dans les diff&amp;eacute;rentes phases du processus. Pour les organisations qui l&amp;rsquo;exp&amp;eacute;rimentent depuis plusieurs ann&amp;eacute;es, les pratiques d&amp;rsquo;&amp;eacute;valuation en phase conception et en phase gestion commencent &amp;agrave; s&amp;rsquo;inst

In [ ]:
clean()

In [57]:
db_df[db_df.idmot == 10675]

,idmot,motcol,niveau,identifiant,resume
5340,10675,Caractère et littérature nationale en Italie (...,titre_article,6,;


In [53]:
db_df = pd.read_csv("mot.csv", sep=";",encoding="utf8", names =["idmot","motcol","niveau","identifiant","resume"])
db_df.dropna()

,idmot,motcol,niveau,identifiant,resume
4672,10007,Note de recherche-Le rôle de l’entrepreneuriat...,titre_article,6,&lt;p style=&quot;text-align:justify; margin-b...
4673,10008,"Transformations organisationnelles, développem...",titre_article,6,&lt;p&gt;&lt;strong&gt;R&amp;eacute;sum&amp;ea...
4674,10009,Comment la digitalisation du don participatif ...,titre_article,6,&lt;p&gt;La recomposition des &amp;eacute;cosy...
4675,10010,La résilience urbaine inclusive,titre_article,6,&lt;p&gt;La tendance mondiale de l&amp;#39;urb...
4676,10011,La territorialisation comme condition de durab...,titre_article,6,&lt;p&gt;R&amp;eacute;sum&amp;eacute;&lt;/p&gt...
...,...,...,...,...,...
6482,11817,Les effets positifs de l'usage des Technologie...,titre_article,6,&lt;p&gt;Les entreprises marocaines commencent...
6483,11818,&Eacute;valuation d'outils et outils d'&eacute...,titre_article,6,&lt;p&gt;&lt;strong&gt;R&amp;eacute;sum&amp;ea...
6484,11819,L&rsquo;analyse des processus m&eacute;tier da...,titre_article,6,&lt;h3&gt;R&amp;eacute;sum&amp;eacute;&amp;nbs...
6485,11820,Quelles &eacute;valuations num&eacute;riques p...,titre_article,6,&lt;p&gt;R&amp;eacute;sum&amp;eacute; : La num...
